# Simple Test between NumPy and Numba

$$
x = \exp(-\Gamma_s d)
$$

In [1]:
import numba
import cython
import numexpr
import numpy as np

%load_ext cython

In [2]:
from empymod import filters
from scipy.constants import mu_0       # Magn. permeability of free space [H/m]
from scipy.constants import epsilon_0  # Elec. permittivity of free space [F/m]

res = np.array([2e14, 0.3, 1, 50, 1])             # nlay
freq = np.arange(1, 201)/20.                    # nfre
off = np.arange(1, 101)*1000                      # noff
lambd = filters.key_201_2009().base/off[:, None]  # nwav

aniso = np.array([1, 1, 1.5, 2, 1])
epermH = np.array([1, 80, 9, 20, 1])
epermV = np.array([1, 40, 9, 10, 1])
mpermH = np.array([1, 1, 3, 5, 1])

etaH = 1/res + np.outer(2j*np.pi*freq, epermH*epsilon_0)
etaV = 1/(res*aniso*aniso) + np.outer(2j*np.pi*freq, epermV*epsilon_0)
zetaH = np.outer(2j*np.pi*freq, mpermH*mu_0)

Gam = np.sqrt((etaH/etaV)[:, None, :, None] * (lambd*lambd)[None, :, None, :] + (zetaH*etaH)[:, None, :, None])

## NumPy

Numpy version to check result and compare times

In [3]:
def test_numpy(lGam, d):
    return np.exp(-lGam*d)

## Numba @vectorize

This is exactly the same function as with NumPy, just added the @vectorize decorater.

In [4]:
@numba.vectorize('c16(c16, f8)')
def test_numba_vnp(lGam, d):
    return np.exp(-lGam*d)

@numba.vectorize('c16(c16, f8)', target='parallel')
def test_numba_v(lGam, d):
    return np.exp(-lGam*d)

## Numba @njit

In [5]:
@numba.njit
def test_numba_nnp(lGam, d):
    out = np.empty_like(lGam)
    for nf in numba.prange(lGam.shape[0]):
        for no in numba.prange(lGam.shape[1]):
            for ni in numba.prange(lGam.shape[2]):
                out[nf, no, ni] = np.exp(-lGam[nf, no, ni] * d)
    return out
                    
@numba.njit(nogil=True, parallel=True)
def test_numba_n(lGam, d):
    out = np.empty_like(lGam)
    for nf in numba.prange(lGam.shape[0]):
        for no in numba.prange(lGam.shape[1]):
            for ni in numba.prange(lGam.shape[2]):
                out[nf, no, ni] = np.exp(-lGam[nf, no, ni] * d)
    return out

## Run comparison for a small and a big matrix

In [6]:
lGam = Gam[:, :, 1, :]
d = 100

# Output shape
out_shape = (freq.size, off.size, filters.key_201_2009().base.size)

print(' Shape Test Matrix    ::', out_shape, '; total # elements:: '+str(freq.size*off.size*filters.key_201_2009().base.size))
print('------------------------------------------------------------------------------------------')

print(' NumPy                ::  ', end='')
# Get NumPy result for comparison
numpy_result = test_numpy(lGam, d)
# Get runtime
%timeit test_numpy(lGam, d)

print(' Numba @vectorize     ::  ', end='')
# Ensure it agrees with NumPy
numba_vnp_result = test_numba_vnp(lGam, d)
if not np.allclose(numpy_result, numba_vnp_result, atol=0, rtol=1e-10):
    print('\n * FAIL, DOES NOT AGREE WITH NumPy RESULT!')
# Get runtime
%timeit test_numba_vnp(lGam, d)

print(' Numba @vectorize par ::  ', end='')
# Ensure it agrees with NumPy
numba_v_result = test_numba_v(lGam, d)
if not np.allclose(numpy_result, numba_v_result, atol=0, rtol=1e-10):
    print('\n * FAIL, DOES NOT AGREE WITH NumPy RESULT!')
# Get runtime
%timeit test_numba_v(lGam, d)

print(' Numba @njit          ::  ', end='')
# Ensure it agrees with NumPy
numba_nnp_result = test_numba_nnp(lGam, d)
if not np.allclose(numpy_result, numba_nnp_result, atol=0, rtol=1e-10):
    print('\n * FAIL, DOES NOT AGREE WITH NumPy RESULT!')
# Get runtime
%timeit test_numba_nnp(lGam, d)

print(' Numba @njit      par ::  ', end='')
# Ensure it agrees with NumPy
numba_n_result = test_numba_n(lGam, d)
if not np.allclose(numpy_result, numba_n_result, atol=0, rtol=1e-10):
    print('\n * FAIL, DOES NOT AGREE WITH NumPy RESULT!')
# Get runtime
%timeit test_numba_n(lGam, d)

 Shape Test Matrix    :: (200, 100, 201) ; total # elements:: 4020000
------------------------------------------------------------------------------------------
 NumPy                ::  305 ms ± 6.26 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
 Numba @vectorize     ::  280 ms ± 13.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
 Numba @vectorize par ::  236 ms ± 34.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
 Numba @njit          ::  248 ms ± 11.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
 Numba @njit      par ::  117 ms ± 704 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
from empymod import versions
versions('HTML', add_pckg=[cython, numba], ncol=5)